In [ ]:
!pip install streamlit
!pip install pyvi

In [ ]:
%%writefile demo.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pyvi import ViTokenizer
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
model = AutoModelForSeq2SeqLM.from_pretrained("MinhViet/Bartpho").to(device)

st.markdown("""
    <style>
    .center-text {
        text-align: center;
    }
    </style>
    """, unsafe_allow_html=True)

st.markdown("<h1 class='center-text'>Abstractive text summarization</h1>", unsafe_allow_html=True)

if 'submitted_question' not in st.session_state:
    st.session_state.submitted_question = ''

question = st.text_input("Nhập văn bản", st.session_state.submitted_question)
segment_question = ViTokenizer.tokenize(question)

confirm_button = st.button("Xác nhận")

if confirm_button and question:
    st.session_state.submitted_question = segment_question

if st.session_state.submitted_question:
    st.write("Đoạn tóm tắt cho văn bản của bạn:")
    encoded_input = tokenizer(st.session_state.submitted_question, return_tensors="pt").to(device)
    output = model.generate(input_ids=encoded_input['input_ids'], attention_mask=encoded_input['attention_mask'], max_length=512, num_beams=5)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    clean_response = response.replace('_', ' ')
    st.code(clean_response)

In [ ]:
! wget -q -O - ipv4.icanhazip.com

In [ ]:
! streamlit run demo.py & npx localtunnel --port 8501